<a href="https://colab.research.google.com/github/priyankachahal/student_research/blob/master/ecg_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import os
import sys
import glob

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [0]:
BASE_PATH = '/content/drive/My Drive/Final Project/dataset/ecg-bg/data/ECG'
DATA_FRAME_FILE = '/content/drive/My Drive/Final Project/dataset/ecg-bg/ecg_reading_data_frame.csv'

In [0]:
def extract_filename_from_path(path):
  return (os.path.split(path)[1])

In [0]:
def get_input_dataframe():
  use_data_frame_file = True
  if use_data_frame_file:
      return pd.read_csv(DATA_FRAME_FILE, engine='python')
  else:
    all_files = glob.glob(BASE_PATH + '/*.csv')
    li_df = []
    counter = 0
    for filename in all_files:
      df = pd.read_csv(filename, usecols=[1], engine='python')
      df = df.transpose();
      df.insert(0, 'fn', extract_filename_from_path(filename))
      li_df.append(df)
      counter = counter + 1
      if counter % 10 == 0:
        print(counter)
    frame = pd.concat(li_df, axis=0, ignore_index=True)
    return frame

In [0]:
df = get_input_dataframe()

In [0]:
#df.to_csv('/content/drive/My Drive/Final Project/dataset/ecg-bg/ecg_reading_data_frame.csv', index = None, header = True)

In [7]:
df

,fn,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,51638,51639,51640,51641,51642,51643,51644,51645,51646,51647,51648,51649,51650,51651,51652,51653,51654,51655,51656,51657,51658,51659,51660,51661,51662,51663,51664,51665,51666,51667,51668,51669,51670,51671,51672,51673,51674,51675,51676,51677
0,956_1.csv,65.347867,197.594708,324.101847,442.088797,549.664593,645.690505,729.652795,801.538240,861.714928,910.828875,949.728682,979.413844,1000.980607,1015.557819,1024.252011,1028.107174,1028.075596,1025.006599,1019.646702,1012.639086,1004.531567,995.791611,986.809541,977.898778,969.309534,961.243087,953.854909,947.253553,941.501905,936.623289,932.612094,929.441478,927.063345,925.407424,924.384784,923.892268,923.814936,924.026399,924.393665,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,869_1.csv,66.426777,197.459011,322.547138,438.949837,544.844439,639.183575,721.557323,792.061721,851.174101,899.639589,938.379735,968.421325,990.835485,1006.689942,1017.013977,1022.769557,1024.831403,1023.976868,1020.886100,1016.152942,1010.295466,1003.759560,996.918403,990.069979,983.438230,977.178569,971.383565,966.093299,961.309079,957.001899,953.117879,949.583110,946.307658,943.197671,940.174581,937.185870,934.207398,931.242642,928.308887,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,961_1.csv,110.166982,248.791284,381.362293,505.035991,617.934959,718.991685,807.796720,884.458442,949.481484,1003.659543,1047.980323,1083.541646,1111.471864,1132.858650,1148.696896,1159.857496,1167.076936,1170.963649,1172.011488,1170.619114,1167.116758,1161.794774,1154.925186,1146.767598,1137.565440,1127.547618,1116.935902,1105.951365,1094.815753,1083.740879,1072.909030,1062.463784,1052.518497,1043.165678,1034.475999,1026.493527,1019.233323,1012.678957,1006.778713,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,403_2.csv,58.463016,188.841826,313.529000,429.856278,536.054785,631.103912,714.588500,786.565895,847.446357,897.891521,938.734801,970.913712,995.409459,1013.206818,1025.275249,1032.553148,1035.924424,1036.191974,1034.059789,1030.133299,1024.930198,1018.883073,1012.331902,1005.522724,998.622898,991.745208,984.968998,978.357241,971.971480,965.879150,960.149018,954.838831,949.981872,945.580492,941.608517,938.015870,934.729726,931.650147,928.653269,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,863_2.csv,55.497380,188.700609,316.456256,435.968942,545.309203,643.283779,729.315546,803.327841,865.633658,916.839396,957.767275,989.388614,1012.762308,1028.977797,1039.102800,1044.142529,1045.020528,1042.581553,1037.604444,1030.808588,1022.843696,1014.267896,1005.532573,996.986900,988.894128,981.446259,974.772754,968.938721,963.935608,959.686194,956.070532,952.948946,950.174421,947.609079,945.142914,942.703307,940.253612,937.784100,935.297391,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2233,608_2.csv,77.991345,212.268938,340.916052,461.208055,571.312641,670.131533,757.164067,832.386327,896.141522,949.042815,991.891513,1025.608035,1051.174695,1069.593461,1081.858500,1088.937868,1091.757477,1091.183583,1088.00